# Before you get started

Wigle.net requires the user to authenticate for accessing the API. The authentication is done via the Authentication Header.

You can access the values for the Authentication Heder at https://wigle.net/account.

To learn more about the Wigle.net API, visit https://api.wigle.net/swagger#/.

In this project API endpoint `/api/v2/network/search` is used. It can is a generic API request that can be configured with many parameters and return a lot of data that we do not need.

The values for the Authentication Header (`Auth_Name` and `Auth_Password`) can be found in the Wigle account (API) settings. They are stored in the `env.py` file which is located in the same directory as this Jupyter notebook. `env.py` is and should remain in the `.gitignore` file to prevent credentials from being published.
```bash
cat env.py
Auth_Name = "..."
Auth_Password = "..."
```

## Probe request data collection
To collect the probe requests we use the pip package `probequest`. It was released together with the paper "[YOUR MOBILE PHONE IS A TRAITOR! – RAISING AWARENESS ON UBIQUITOUS PRIVACY ISSUES WITH SASQUATCH](https://brambonne.com/docs/bonne14sasquatch.pdf)" by Bram Bonné, Peter Quax and Wim Lamotte in 2014.

To install the package either run:
```bash
pip3 install --upgrade probequest
```
or get it directly from GitHub:
```bash
git clone -b main https://github.com/SkypLabs/probequest.git
cd probequest
pip3 install --upgrade .
```

Before the package can be used, your device needs to have a wireless network interface in monitor mode. How to archive this can be OS dependent. One option would be `iw` on Linux. In my chase:
```bash
sudo ip link set wlp0s20f0u4u1 down
sudo iw wlp0s20f0u4u1 set monitor none
sudo ip link set wlp0s20f0u4u1 up
```
Here `wlp0s20f0u4u1` is the name of the wireless network interface.

Probequest gives two useful parameters:
* `-o` which specifies an output file in CSV format (separation by `;`)
* `--fake` which generates arbitrary. This can be used for testing and demonstration purposes. You have to specify an interface, but it does not have to be in monitor mode.

Additionally probequest can be configured to filter for specific SSIDs. We won't be using this feature an will filter if necessary using python.

To start the data collection run the `probequest.sh` script. In the script you may select to either use fake data or real data. Keep in mind that the collection of real data may fall under the GDPR. The script runs until it is stopped by the user. When the script finished, it writes the data to the CSV file `wifiData.csv`.

Configure the script to your needs:
* `interface` is the name of the wireless network interface
* `--fake` or not, depending on if you want to use fake data or not

API requests on wigle.net are limited per day for normal users. I didn't manage to find the exact number of request per day and believe that it is not publicly disclosed. My rough estimation would be lower than 50. I personally managed to hit the API limit while building and debugging this project. 
wigle.net state on the account page, that they may increase your API limit if you contact them. 

I have tried this, stating my use case, educational strictly non-commercial use and that I am a student. wigle answered quickly and increased my API limit significantly so that I did have problems since.
My best regards to the wigle.net team for their quick and helpful support.

In [1]:
import env

## Variables
These variable are listed here for better configuration.

In [2]:
# Path to the csv file generated by probequest.sh
csv_file = "wifiData.csv"

# Colors used to represent each unique MAC address that sent a probe request on the geographic map
# These values are a partial selection of the colorbrewer2.org qualitative color scheme https://colorbrewer2.org/#type=qualitative&scheme=Accent&n=3
hexColors = [
    '#1f78b4',
    '#e31a1c',
    '#ff7f00',
    '#6a3d9a',
    '#b15928',
]

# Mean coordinates of the country that is displayed on the map
# When changing to other countries, you can use this table from google developers to find the mean coordinates:
# https://developers.google.com/public-data/docs/canonical/countries_csv
map_meanCoordinates = [51.165691, 10.451526] # Germany
map_zoomLevel = 7 # Change according to the size of the country

# If you what to change the country, you also should change the query parameter for wigle.net
query_country = "DE" # Germany

# I didn't find official documentation for the list of country codes but I think they are fairly standard.
# Additionally you can test your queries with the interactive API: https://api.wigle.net/swagger#/Network%20search%20and%20information%20tools/search_2 


### Imports


In [3]:
import requests
from dataclasses import dataclass

import pandas as pd
from typing import List

from pprint import pprint

from requests.auth import HTTPBasicAuth

import folium
import folium.plugins

import webbrowser

### Data Classes

In [4]:
@dataclass
class MAC_SSID:
    MAC: str
    SSID: str

@dataclass
class unique_MAC:
    MAC: str
    list_SSID: list[str]

    def __repr__(self):
        return f'{self.MAC} {self.list_SSID}\n'
    
@dataclass
class Information:
    lat: float
    long: float
    country: str
    region: str
    city: str
    housenumber: str
    postalcode: str
    encryption: str
    channel: int
    bSSID: str
    SSID: str

@dataclass
class SSID_Information:
    ssid: str
    information : Information

@dataclass
class MAC_SSID_Information:
    mac: str
    ssid_information_list: list[SSID_Information]

### Read CSV

Data is formated like: `time;MAC;OUI;SSID`
```csv
1686297722.6456103;5c:27:c9:50:03:ce;Unknown OUI;UPC8874773
1686297723.6474442;b2:a2:24:52:11:96;Unknown OUI;UPC2537522
1686297724.6511312;4index:59:d2:61:5c:20;Unknown OUI;UPC6771688
1686297725.654444;b9:59:1d:1b:cc:09;Unknown OUI;Bbox-E47004
1686297725.654445;b9:59:1d:1b:cc:09;Unknown OUI;HelloWorld
```


In [5]:
data_import = pd.read_csv(csv_file,     # Read first pandas DataFrame column
                           usecols = [1,3], sep=';', index_col=False, header=None, skiprows=0)

#### Extract data MAC and SSID

In [6]:
list_MAC_SSID:List[MAC_SSID] = []

import_list = data_import.values.tolist()

for row in import_list:
    list_MAC_SSID.append(MAC_SSID(MAC=row[0], SSID=row[1]))

In [7]:
list_unique_MAC:List[unique_MAC] = []

# convert list_MAC_SSID to dict

dict_uniqueMAC = {}
for mac_ssid in list_MAC_SSID:
    if mac_ssid.MAC not in dict_uniqueMAC:
        dict_uniqueMAC[mac_ssid.MAC] = [mac_ssid.SSID]
    else:
        dict_uniqueMAC[mac_ssid.MAC].append(mac_ssid.SSID)

# convert dict to list_unique_MAC
for key in dict_uniqueMAC:
    list_unique_MAC.append(unique_MAC(key, dict_uniqueMAC[key]))

### Wigle API request
https://api.wigle.net/api/v2/network/search?onlymine=false&freenet=false&paynet=false&ssid=eduroam&country=DE

Parameters:
```h
onlymine: false
freenet: false
paynet: false
ssid: {SSID}
country: DE -- current limitation to Germany to reduce the number of results
```

In [8]:
auth = HTTPBasicAuth(env.Auth_Name, env.Auth_Password)

In [11]:
list_mac_ssid_information:List[MAC_SSID_Information] = []

dict_mac_ssid_information = {}

print(list_unique_MAC)
for unique_mac in list_unique_MAC:
    for ssid_per_mac in unique_mac.list_SSID:
        print(f'Requesting Wigle API for SSID: {ssid_per_mac}')
        response = requests.get('https://api.wigle.net/api/v2/network/search', params={
            'onlymine': 'false',
            'freenet': 'false',
            'paynet': 'false',
            'ssid': ssid_per_mac,
            'country': "DE",
            } , auth=auth)
        jsonResponse = response.json()

        print(jsonResponse)
        
        if jsonResponse['totalResults'] == 0:
            print(f'No results found for SSID: {ssid_per_mac}')
            break
        
        for index, result in enumerate(jsonResponse['results']):
            if unique_mac.MAC not in dict_mac_ssid_information:
                dict_mac_ssid_information[unique_mac.MAC] = [
                    SSID_Information(
                        ssid=ssid_per_mac,
                        information=Information(
                            lat=jsonResponse['results'][index]['trilat'],
                            long=jsonResponse['results'][index]['trilong'],
                            country=jsonResponse['results'][index]['country'],
                            region=jsonResponse['results'][index]['region'],
                            city=jsonResponse['results'][index]['city'],
                            housenumber=jsonResponse['results'][index]['housenumber'],
                            postalcode=jsonResponse['results'][index]['postalcode'],
                            encryption=jsonResponse['results'][index]['encryption'],
                            channel=jsonResponse['results'][index]['channel'],
                            bSSID=jsonResponse['results'][index]['netid'],
                            SSID=jsonResponse['results'][index]['ssid'],
                        )
                    )
                ]
            else:
                dict_mac_ssid_information[unique_mac.MAC].append(
                    SSID_Information(
                        ssid=ssid_per_mac,
                        information=Information(
                            lat=jsonResponse['results'][index]['trilat'],
                            long=jsonResponse['results'][index]['trilong'],
                            country=jsonResponse['results'][index]['country'],
                            region=jsonResponse['results'][index]['region'],
                            city=jsonResponse['results'][index]['city'],
                            housenumber=jsonResponse['results'][index]['housenumber'],
                            postalcode=jsonResponse['results'][index]['postalcode'],
                            encryption=jsonResponse['results'][index]['encryption'],
                            channel=jsonResponse['results'][index]['channel'],
                            bSSID=jsonResponse['results'][index]['netid'],
                            SSID=jsonResponse['results'][index]['ssid'],
                        )
                    )
                )

# Convert dict_mac_ssid_information to list_mac_ssid_information
for key in dict_mac_ssid_information:
    list_mac_ssid_information.append(
        MAC_SSID_Information(
            mac=key,
            ssid_information_list=dict_mac_ssid_information[key]
        )
    )

[21:09:81:b1:ab:cf ['UPC9476371']
, 13:10:95:d2:9b:eb ['GetYourOwn']
, da:c8:8e:e8:92:08 ['visitors']
]
Requesting Wigle API for SSID: UPC9476371
{'success': True, 'totalResults': 0, 'first': 1, 'last': 0, 'resultCount': 0, 'results': [], 'searchAfter': None, 'search_after': None}
No results found for SSID: UPC9476371
Requesting Wigle API for SSID: GetYourOwn
{'success': True, 'totalResults': 89, 'first': 1, 'last': 89, 'resultCount': 89, 'results': [{'trilat': 51.22760391, 'trilong': 6.78995419, 'ssid': 'GETYOUROWN', 'qos': 0, 'transid': '20060317-00000', 'firsttime': '2001-01-01T00:00:00.000Z', 'lasttime': '2006-03-17T16:00:00.000Z', 'lastupdt': '2006-03-17T14:00:00.000Z', 'netid': '00:03:C9:7D:FA:24', 'name': None, 'type': None, 'comment': None, 'wep': 'Y', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 11, 'encryption': 'wep', 'country': 'DE', 'region': 'Nordrhein-Westfalen', 'road': 'Oststraße', 'city': 'Düsseldorf', 'housenumber': '10

### Map Coord and Information on geo Map

In [33]:

# Folium Map Creation
map = folium.Map(
    map_meanCoordinates,
    zoom_start=map_zoomLevel,
    )

# Create a separate marker cluster for each MAC
for cIndex, mac_ssid_information in enumerate(list_mac_ssid_information):
    locations = folium.plugins.MarkerCluster(
        disableClusteringAtZoom=0,
    ).add_to(map)

    for ssid_information in mac_ssid_information.ssid_information_list:
        folium.vector_layers.CircleMarker(
            location=[ssid_information.information.lat, ssid_information.information.long],
            popup=str(f"""
                From MAC: <b>{mac_ssid_information.mac}</b> <br>
                <ul>
                    <li>SSID: <b>{ssid_information.information.SSID}</b> <br> </li>
                    <li>MAC: {ssid_information.information.bSSID} <br> </li>
                    <li>Encryption: {ssid_information.information.encryption} <br> </li>
                    <li>Channel: {ssid_information.information.channel} <br> </li>
                </ul>
                """
                ),
            radius=15,
            color=hexColors[cIndex%len(hexColors)],
            fill=True,
            fill_color=hexColors[cIndex%len(hexColors)],
            fill_opacity=0.2, 
            weight=1,
            opacity=1,
        ).add_to(locations)

# The following code is for generating the legend starting with --- and ending with ---. This code was inspired by ChatGPT. The unaltered code is linked in the chatGPT.md file.
# ---
legend_html = '''
     <div style="position: fixed; 
                 top: 50px; right: 50px; width: 200px; height: 90px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;">
         &nbsp; <b> Legend </b> <br>
'''

for cIndex, mac_ssid_information in enumerate(list_mac_ssid_information):
    legend_html += f'&nbsp; <i class="fa fa-circle fa-1x" style="color:{hexColors[cIndex%len(hexColors)]}"></i> {mac_ssid_information.mac}<br>'

legend_html += '''
     </div>
'''

# Add the legend to the map
map.get_root().html.add_child(folium.Element(legend_html))
# ---

map.add_child(locations)


# Save the file with the name "map_YYYY-MM-DD_HH-MM-SS.html"
from datetime import datetime
now = datetime.now()
map.save('map_' + now.strftime("%Y-%m-%d_%H-%M-%S") + '.html')


## Output

In [34]:
webbrowser.open('map.html')

pprint(list_mac_ssid_information)

[MAC_SSID_Information(mac='13:10:95:d2:9b:eb',
                      ssid_information_list=[SSID_Information(ssid='GetYourOwn',
                                                              information=Information(lat=51.22760391,
                                                                                      long=6.78995419,
                                                                                      country='DE',
                                                                                      region='Nordrhein-Westfalen',
                                                                                      city='Düsseldorf',
                                                                                      housenumber='10',
                                                                                      postalcode='40211',
                                                                                      encryption='wep',
                               